In [ ]:
import os
os.environ["HOME"]

In [ ]:
import tensorflow as tf
print(tf.__version__)
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
import numpy as np
def one_hot_encoder(s):
    pep_num = ' '.join([str(ints) for ints in range(20)])
    pep_ref = 'A C D E F G H I K L M N P Q R S T V W Y'
    d = dict(zip(pep_ref.split(' '), pep_num.split(' ')))
    
    x = np.zeros((len(d), len(s)))
    x[[int(d[c]) for c in s], range(len(s))] = 1
    return x

In [ ]:
import pandas as pd
folder = '/path_for_peptide_sequences/pepide_sequences_with_class_file'
well_4000 = pd.read_csv('{}/Transwell_4000_total.csv'.format(folder), sep = ",")
chip_4000 = pd.read_csv('{}/Chip2um_4000_total.csv'.format(folder), sep = ",")
invivo_4000 = pd.read_csv('{}/Invivo_4000_total.csv'.format(folder), sep = ",")

In [ ]:
#Sequences to One-hot encoding vector for CNN
pep_seq = invivo_4000['#pseq'].tolist()
pep_class = invivo_4000['Class'].tolist()
X_seq = [one_hot_encoder(x) for x in pep_seq]
X_seq = np.transpose(np.asarray(X_seq), (0, 2, 1))
X_seq = X_seq.astype('float32')
Y_class = np.array(pep_class)

In [ ]:
print('Length of data:', X_seq.shape[0])
print('Length of peptides:', X_seq.shape[1])
print('Length of amino acids:', X_seq.shape[2])

In [ ]:
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras import models
import numpy as np
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import accuracy_score
from tensorflow.keras import optimizers
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, precision_recall_curve,average_precision_score, confusion_matrix, f1_score,matthews_corrcoef
from inspect import signature

In [ ]:
#Model path
import os
model_path = os.getcwd()
model_path = os.path.join(model_path,'models')
os.makedirs(model_path, exist_ok = True)
print(model_path)

In [ ]:
#Hyperparameter : manually

BATCH_SIZE = 16
PEP_length = 12
AA_type  = 20
CLASS_NAMES = ['enriched','not-enriched']
params = [['CONV', 400, 3, 1],
          ['DROP', 0.7],
          ['POOL', 2, 1],
          ['FLAT'],
          ['DENSE', 50]]
activation_func = 'relu'
regularizer_params = None

In [ ]:
model_version = 1
dict_history = {}
#s225 (steps per epoch) * 16(batch size) = 3600(training data set number)
steps_per_epoch = 225
Epochs = 50

In [ ]:
import numpy as np
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import accuracy_score
from tensorflow.keras import optimizers
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, precision_recall_curve,average_precision_score, confusion_matrix, f1_score,matthews_corrcoef
from inspect import signature
split_data = ShuffleSplit(n_splits = 10, train_size = None, test_size = 0.1, random_state=1)

acc_per_fold = []
loss_per_fold = []
val_acc_per_fold = []
val_loss_per_fold = []
roc_per_fold = []
roc_fpr = []
roc_tpr = []
precision_fold = []
recall_fold = []
ave_precision_fold = []
n_iter =0
fold_num = []

for train_idx, test_idx in split_data.split(X_seq, Y_class):
    X_train = np.array(X_seq[train_idx])
    X_test = np.array(X_seq[test_idx])
    y_train = np.array(Y_class[train_idx])
    y_test = np.array(Y_class[test_idx])
    model = models.Sequential()
    model.add(layers.Conv1D(filters = params[0][1],kernel_size = params[0][2],strides = params[0][3],activation=activation_func, input_shape=(PEP_length,AA_type),kernel_regularizer=regularizer_params, bias_regularizer=regularizer_params,padding='same'))
    model.add(layers.Dropout(rate = params[1][1])) #add noise
    model.add(layers.MaxPool1D(pool_size = params[2][1], strides = params[2][2]))
    model.add(layers.Flatten())
    model.add(layers.Dense(params[4][1], activation=activation_func, kernel_regularizer=regularizer_params,bias_regularizer=regularizer_params))
    model.add(layers.Dense(1, activation='sigmoid')) #'softmax' for N class
    model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.000075), metrics=['acc'] )
    dict_history[model_version] = model.fit(x=X_train,y=y_train,shuffle=True, steps_per_epoch=steps_per_epoch, epochs=Epochs, batch_size = BATCH_SIZE, validation_data=(X_test, y_test), verbose=2)
    history = dict_history[model_version]
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    scores = model.evaluate(X_test, y_test, verbose=0)
    print(f'Score for fold {n_iter+1}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%\n')
    acc_per_fold.append(acc)
    loss_per_fold.append(loss)
    val_acc_per_fold.append(val_acc)
    val_loss_per_fold.append(val_loss)
    arr_acc = np.array(acc_per_fold)
    arr_loss = np.array(loss_per_fold)
    arr_val_acc = np.array(val_acc_per_fold)
    arr_val_loss = np.array(val_loss_per_fold)
    epochs = range(1, len(acc) + 1)
    pred = model.predict(x=X_test)
    fpr, tpr, thresholds = roc_curve(y_test, pred)
    roc_fpr.append(fpr)
    roc_tpr.append(tpr)
    roc_auc = auc(fpr, tpr)
    roc_per_fold.append(roc_auc)
    precision, recall, thresholds = precision_recall_curve(y_test, pred)
    average_precision = average_precision_score(y_test, pred)
    precision_fold.append(precision)
    recall_fold.append(recall)
    ave_precision_fold.append(average_precision)
    globals()['model_fname_{}'.format(n_iter+1)] = f'BBBphagedisplay_TEST_{n_iter+1}_{model_version:03}.h5'
    model_spath = os.path.join(model_path,f'model_fname_{n_iter+1}')
    model.save(model_spath)
    fold_num.append(n_iter+1)
    n_iter += 1
    epochs = range(1, len(acc) + 1)
    fig = plt.figure(figsize=(10.5,3.5))
    plt.plot(epochs, acc, 'bo', label='Training acc')
    plt.plot(epochs, val_acc, 'r', alpha=0.7, label='Validation acc')
    plt.title(f'Training and validation accuracy of fold {n_iter+1}')
    plt.legend()
    fig = plt.figure(figsize=(10.5,3.5))
    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'r', alpha=0.7, label='Validation loss')
    plt.title(f'Training and validation loss of fold {n_iter+1}')
    plt.legend()
    plt.show()   
    pred = model.predict(x=X_test)
    fpr, tpr, thresholds = roc_curve(y_test, pred)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, color='darkorange',
             lw=2, label='ROC curve (area - %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc='lower right')
    plt.show()
    plt.cla()
    plt.clf()
    precision, recall, thresholds = precision_recall_curve(y_test, pred)
    average_precision = average_precision_score(y_test, pred)
    step_kwargs = ({'step': 'post'}
                   if 'step' in signature(plt.fill_between).parameters
                   else {})
    plt.step(recall, precision, color='navy', alpha=0.2, where='post', label='Avg. Precision: {0:0.2f}'.format(average_precision))
    plt.fill_between(recall, precision, alpha=0.2, color='navy', **step_kwargs)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.legend(loc='lower right')
    plt.title(f'Precision recall curve and ROC curve of fold {n_iter+1}')
    plt.show()
    plt.cla()
    plt.clf()

In [ ]:
#Model save
model_fname = f'BBBphagedisplay_TEST_{model_version:03}.cv.h5'
model_spath = os.path.join(model_path, model_fname)
print(model_spath)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from cycler import cycler
import matplotlib.lines as mlines
num_plots = 10
n_iter = 0

plt.rcParams["font.size"] = 16
fig, ax = plt.subplots(figsize=(10.5,5.5))
ax.set_prop_cycle('color',[plt.cm.jet(i) for i in np.linspace(0, 1, num_plots)])
b_o = mlines.Line2D([], [], color='black', marker='o', linestyle='None', markersize=8)
b_line = mlines.Line2D([], [], color='black', linestyle='solid')
for i in range(n_iter, num_plots):
    plt.plot(epochs, arr_acc[i], 'o', markersize = 5.5, alpha= 0.8)
    i += 1
for i in range(n_iter, num_plots):
    plt.plot(epochs, arr_val_acc[i], alpha= 0.8, label= f'Fold {i+1}')
    i += 1
legend = plt.legend(loc='upper right', bbox_to_anchor=(1.15, 1), frameon = True, fontsize = 11)
art_legend = plt.gca().add_artist(legend)
legend_2 = plt.legend(handles = [b_o, b_line], labels = ['Accuracy', 'Validation accuracy'], loc='lower right', frameon = True, fontsize = 11)
art_legend_2 = plt.gca().add_artist(legend_2)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
num_plots = 10
n_iter = 0

fig, ax = plt.subplots(figsize=(10.5,5.5))
ax.set_prop_cycle('color',[plt.cm.jet(i) for i in np.linspace(0, 1, num_plots)])
b_o = mlines.Line2D([], [], color='black', marker='o', linestyle='None', markersize=8)
b_line = mlines.Line2D([], [], color='black', linestyle='solid')
for i in range(n_iter, num_plots):
    plt.plot(epochs, arr_loss[i], 'o', markersize = 5.5, alpha= 0.8)
    i += 1
for i in range(n_iter, num_plots):
    plt.plot(epochs, arr_val_loss[i], alpha= 0.8, label= f'Fold {i+1}')
    i += 1
legend = plt.legend(loc='upper right', bbox_to_anchor=(1.15, 1), frameon = True, fontsize = 11)
art_legend = plt.gca().add_artist(legend)
legend_2 = plt.legend(handles = [b_o, b_line], labels = ['Loss', 'Validation loss'], loc='lower right', frameon = True, fontsize = 11)
art_legend_2 = plt.gca().add_artist(legend_2)

In [ ]:
num_plots = 10
n_iter = 0

plt.rcParams["font.size"] = 16
fig, ax = plt.subplots(figsize=(7, 7))
ax.set_prop_cycle('color',[plt.cm.jet(i) for i in np.linspace(0, 1, num_plots)])
plt.plot([0, 1], [0, 1], color='navy', linewidth=1, linestyle='--')
for i in range(n_iter, num_plots):
    plt.plot(roc_fpr[i], roc_tpr[i], linewidth=1, label=f'ROC curve (area - %0.2f) of fold {i+1}' % roc_per_fold[i])
    i += 1
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()
plt.cla()
plt.clf()

In [ ]:
num_plots = 10
n_iter = 0

fig, ax = plt.subplots(figsize=(8.5, 8.5))
ax.set_prop_cycle('color',[plt.cm.jet(i) for i in np.linspace(0, 1, num_plots)])
for i in range(n_iter, num_plots):
    step_kwargs = ({'step': 'post'}
                   if 'step' in signature(plt.fill_between).parameters
                   else {})
    plt.step(recall_fold[i], precision_fold[i], color='navy', alpha=0.1, where='post', label='Avg. Precision of each fold : {0:0.2f}'.format(ave_precision_fold[i]))
    plt.fill_between(recall_fold[i], precision_fold[i], alpha=0.05, color='navy', **step_kwargs)
    i += 1
plt.xlabel('Recall', fontsize = 20)
plt.ylabel('Precision', fontsize = 20)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xticks(fontsize = 20)
plt.yticks(fontsize = 20)
legend = plt.legend(loc='upper right', bbox_to_anchor=(1.6, 1), frameon = False, fontsize = 13)
art_legend = plt.gca().add_artist(legend)
plt.title(f'Precision-Recall curve of {i} folds', fontsize = 20)
plt.show()
plt.cla()
plt.clf()

In [ ]:
num_plots = 10
n_iter = 0

fig, ax = plt.subplots(figsize=(8.5, 8.5))
ax.set_prop_cycle('color',[plt.cm.jet(i) for i in np.linspace(0, 1, num_plots)])
plt.plot([0, 1], [0, 1], color='navy', linewidth=1, linestyle='--')
for i in range(n_iter, num_plots):
    plt.plot(roc_fpr[i], roc_tpr[i], linewidth=1, label=f'ROC curve (area - %0.2f) of fold {i+1}' % roc_per_fold[i])
    i += 1
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize = 20)
plt.ylabel('True Positive Rate', fontsize = 20)
plt.xticks(fontsize = 20)
plt.yticks(fontsize = 20)
legend = plt.legend(loc='upper right', bbox_to_anchor=(1.6, 1), frameon = True, fontsize = 13)
art_legend = plt.gca().add_artist(legend)
plt.show()
plt.cla()
plt.clf()

In [ ]:
import pandas as pd

well_pep = well_4000['#pseq'].tolist()
chip_pep = chip_4000['#pseq'].tolist()
well_class = well_4000['Class'].tolist()
chip_class = chip_4000['Class'].tolist()
well_encode = [one_hot_encoder(x) for x in well_pep]
well_encode = np.transpose(np.asarray(well_encode), (0, 2, 1))
chip_encode = [one_hot_encoder(x) for x in chip_pep]
chip_encode = np.transpose(np.asarray(chip_encode), (0, 2, 1))

In [ ]:
model_fit = keras.models.load_model("/path_to_models/models/model_fname_num/")
well_score = model_fit.predict(well_encode)
chip_score = model_fit.predict(chip_encode)
well_score_flat = well_score.flatten()
chip_score_flat = chip_score.flatten()

In [ ]:
model_fit.fit(well_encode, well_score_flat)

In [ ]:
model_fit.fit(chip_encode, chip_score_flat)

In [ ]:
from sklearn.metrics import roc_curve, auc, precision_recall_curve,average_precision_score, confusion_matrix, f1_score,matthews_corrcoef
from inspect import signature

plt.rcParams["font.size"] = 16
fpr, tpr, thresholds = roc_curve(well_class, well_score)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, color='darkorange',
         lw=2, label='ROC curve (area = %0.3f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
plt.show()
plt.cla()
plt.clf()

precision, recall, thresholds = precision_recall_curve(
    well_class, well_score
)
average_precision = average_precision_score(well_class, well_score)

step_kwargs = ({'step': 'post'}
               if 'step' in signature(plt.fill_between).parameters
               else {})
plt.step(recall, precision, color='navy', alpha=0.2, where='post',
         label='Avg. Precision : {0:0.3f}'.format(average_precision))
plt.fill_between(recall, precision, alpha=0.2, color='navy', **step_kwargs)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.legend(loc='lower right')
plt.show()
plt.cla()
plt.clf()

In [ ]:
from sklearn.metrics import roc_curve, auc, precision_recall_curve,average_precision_score, confusion_matrix, f1_score,matthews_corrcoef
from inspect import signature

plt.rcParams["font.size"] = 16
fpr, tpr, thresholds = roc_curve(chip_class, chip_score)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, color='darkorange',
         lw=2, label='ROC curve (area = %0.3f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
plt.show()
plt.cla()
plt.clf()


precision, recall, thresholds = precision_recall_curve(
    chip_class, chip_score
)
average_precision = average_precision_score(chip_class, chip_score)

step_kwargs = ({'step': 'post'}
               if 'step' in signature(plt.fill_between).parameters
               else {})
plt.step(recall, precision, color='navy', alpha=0.2, where='post',
         label='Avg. Precision : {0:0.3f}'.format(average_precision))
plt.fill_between(recall, precision, alpha=0.2, color='navy', **step_kwargs)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.legend(loc='lower right')
plt.show()
plt.cla()
plt.clf()